In [32]:
import pandas as pd
from tensorflow.keras import models, layers

In [33]:
# Constant to store the file path
FILE_PATH = "data/balance_sheets_quarterly_processed.csv"

# Read the data from the CSV file
df = pd.read_csv(FILE_PATH)

# Display the first few rows of the dataframe to verify the data
df.head()

,date,symbol,cashAndCashEquivalents,shortTermInvestments,inventory,totalCurrentAssets,propertyPlantEquipmentNet,goodwill,longTermInvestments,otherNonCurrentAssets,...,totalNonCurrentLiabilities,commonStock,retainedEarnings,accumulatedOtherComprehensiveIncomeLoss,othertotalStockholdersEquity,totalStockholdersEquity,minorityInterest,totalDebt,netDebt,marketCap
0,2024-03-31,PKP.WA,176600000.0,0.0,189800000.0,1.127200e+09,6.803700e+09,0.0,51500000.0,49300000.0,...,2.770700e+09,2.239300e+09,49500000.0,776100000.0,0.0,3.064900e+09,0.0,2.895400e+09,2.718800e+09,2.929004e+10
1,2023-12-31,PKP.WA,226200000.0,-10100000.0,200200000.0,1.289400e+09,6.734100e+09,0.0,52800000.0,105200000.0,...,2.784200e+09,2.239300e+09,167600000.0,795600000.0,0.0,3.202500e+09,0.0,2.893800e+09,2.667600e+09,3.264899e+10
2,2023-09-30,PKP.WA,216800000.0,-16900000.0,224200000.0,1.295600e+09,6.698500e+09,0.0,58000000.0,104600000.0,...,2.576200e+09,2.239300e+09,188600000.0,858500000.0,0.0,3.286400e+09,0.0,2.653000e+09,2.436200e+09,3.296250e+10
3,2023-06-30,PKP.WA,174600000.0,-16900000.0,217600000.0,1.212800e+09,6.605600e+09,0.0,57400000.0,116200000.0,...,2.529400e+09,2.239300e+09,196400000.0,867500000.0,0.0,3.303200e+09,0.0,2.650400e+09,2.475800e+09,3.842639e+10
4,2023-03-31,PKP.WA,230300000.0,-16300000.0,219700000.0,1.395700e+09,6.367400e+09,0.0,59000000.0,123600000.0,...,2.550900e+09,2.239300e+09,308800000.0,809500000.0,0.0,3.357600e+09,0.0,2.675800e+09,2.445500e+09,3.585119e+10
